In [129]:
# !pip install tensorflow
#!pip install opencv-python
!pip install keras


In [146]:
import tensorflow as tf
from tensorflow import keras


In [149]:

import numpy as np
import matplotlib.pyplot as plt
#import nn_utils
from pathlib import Path
import pandas as pd
import cv2
from glob import glob
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D
#from tensorflow.keras.utils import np_utils


In [111]:
label_dir = {'colon':1,'endometrium_1':2,'endometrium_2':3,'kidney':4,'liver':5,'lung':6,'lymph_node':7,'pancreas':8,'skin_1':9,'skin_2':10,'small_intestine':11,'spleen':12}

In [90]:
images_dir = r'D:\\mitali\\train\\'
train_data = []
folder_list = os.listdir(images_dir)
for ind,name in enumerate (folder_list):
    images = Path(images_dir+str(name)).glob("*.tif")
    for i in images:
        train_data.append((i,label_dir[name]))


In [91]:
train_data

[(WindowsPath('D:/mitali/train/colon/40043_94144_A_7_3.tif'), 1),
 (WindowsPath('D:/mitali/train/colon/40043_94144_A_8_3.tif'), 1),
 (WindowsPath('D:/mitali/train/colon/40043_94144_A_9_3.tif'), 1),
 (WindowsPath('D:/mitali/train/colon/40044_96798_A_8_3.tif'), 1),
 (WindowsPath('D:/mitali/train/colon/40045_97686_A_7_3.tif'), 1),
 (WindowsPath('D:/mitali/train/colon/40045_97686_A_8_3.tif'), 1),
 (WindowsPath('D:/mitali/train/colon/40046_142250_A_7_3.tif'), 1),
 (WindowsPath('D:/mitali/train/colon/40046_142250_A_8_3.tif'), 1),
 (WindowsPath('D:/mitali/train/colon/40046_142250_A_9_3.tif'), 1),
 (WindowsPath('D:/mitali/train/colon/40048_84862_A_7_3.tif'), 1),
 (WindowsPath('D:/mitali/train/colon/40048_84862_A_8_3.tif'), 1),
 (WindowsPath('D:/mitali/train/colon/40048_84862_A_9_3.tif'), 1),
 (WindowsPath('D:/mitali/train/colon/40051_87105_A_7_3.tif'), 1),
 (WindowsPath('D:/mitali/train/colon/40051_87105_A_8_3.tif'), 1),
 (WindowsPath('D:/mitali/train/colon/40051_87105_A_9_3.tif'), 1),
 (Windo

In [92]:
len(train_data)

199

In [93]:
train_data = pd.DataFrame(train_data,columns=['image','label'],index=None)
train_data

,image,label
0,D:\mitali\train\colon\40043_94144_A_7_3.tif,1
1,D:\mitali\train\colon\40043_94144_A_8_3.tif,1
2,D:\mitali\train\colon\40043_94144_A_9_3.tif,1
3,D:\mitali\train\colon\40044_96798_A_8_3.tif,1
4,D:\mitali\train\colon\40045_97686_A_7_3.tif,1
...,...,...
194,D:\mitali\train\lung\40051_87105_A_3_4.tif,6
195,D:\mitali\train\lung\40052_130536_A_1_4.tif,6
196,D:\mitali\train\lung\40052_130536_A_3_4.tif,6
197,D:\mitali\train\lung\40055_84642_A_2_4.tif,6


In [94]:
train_data['image'] = train_data['image'].apply(lambda x : cv2.imread(str(x)))

In [97]:
train_data['image'][0].shape

(3000, 3000, 3)

ERROR! Session/line number was not unique in database. History logging moved to new session 541


# Data Augmentation to create new data with different orientations

In [98]:
#train_data['image'] = train_data['image'].apply(lambda x : cv2.cvtColor(x, cv2.COLOR_BGR2GRAY))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-98-db95cabe4567>", line 1, in <module>
    train_data['image'] = train_data['image'].apply(lambda x : cv2.cvtColor(x, cv2.COLOR_BGR2GRAY))
  File "D:\Anaconda\lib\site-packages\pandas\core\series.py", line 3848, in apply
    mapped = lib.map_infer(values, f, convert=convert_dtype)
  File "pandas\_libs\lib.pyx", line 2327, in pandas._libs.lib.map_infer
  File "<ipython-input-98-db95cabe4567>", line 1, in <lambda>
    train_data['image'] = train_data['image'].apply(lambda x : cv2.cvtColor(x, cv2.COLOR_BGR2GRAY))
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError:

KeyboardInterrupt: 

In [201]:
split_criteria = round(len(train_data)*0.8)

x_train = train_data['image'][:split_criteria]
y_train = train_data['label'][:split_criteria]
x_test = train_data['image'][split_criteria:]
y_test = train_data['label'][split_criteria:]

In [198]:
x_train = x_train.values.reshape(x_train.shape[0],1)
#x_test = x_test.values.reshape(x_test.shape[0],1)
x_train.shape

(159, 1)

# Training a CNN


In [200]:
model = keras.Sequential()
model.add(layers.Conv2D(6,(3,3),input_shape = x_train.shape,activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2),strides=2))
model.add(Dropout(0.4))
model.add(layers.Conv2D(32,(3,3),padding='valid',activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2),strides=2,padding = 'valid'))
model.add(Dropout(0.6))
model.add(layers.Conv2D(64,(3,3),padding='valid',activation='relu'))
model.add(Dropout(0.8))
model.add(layers.Flatten())
model.add(Dense(6,activation='softmax'))



ValueError: Input 0 of layer conv2d_231 is incompatible with the layer: expected ndim=4, found ndim=3. Full shape received: [None, 159, 1]

In [174]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')


In [175]:
model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))

ValueError: Error when checking input: expected conv2d_22_input to have 4 dimensions, but got array with shape (159, 1)

# Pre-trained model

In [173]:
#Load the InceptionResNetV2 architecture with imagenet weights as base
base_model = tf.keras.applications.InceptionResNetV2(
                     include_top=False,
                     weights='imagenet',
                     input_shape=(3000,3000,3)
                     )

219062272/219055592 [==============================] - 23s 0us/step


In [ ]:
base_model.trainable=False
# For freezing the layer we make use of layer.trainable = False
# means that its internal state will not change during training.
# model's trainable weights will not be updated during fit(),
# and also its state updates will not run.
  
model = tf.keras.Sequential([ 
        base_model,   
        tf.keras.layers.BatchNormalization(renorm=True),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(120, activation='softmax')
    ])

model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train)
